In [22]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
from sklearn.datasets import make_blobs
import mglearn
import sklearn
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler


# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
#style.use('ggplot') or plt.style.use('ggplot')

# Preprocesado y modelado
# ==============================================================================
from sklearn.ensemble import IsolationForest
from sklearn.pipeline import make_pipeline

# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
from sklearn.datasets import make_classification
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.datasets import make_classification
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import MinMaxScaler

#Random global
seed = 12345
rng = np.random.default_rng(seed)  # can be called without a seed
rng.random()
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
df = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\RFECved.csv")
dfNon = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\scaled.csv")
df2 = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\Deoutliers2.csv")
dfR = pd.read_csv (r"C:\Users\White\Desktop\tfg\generados\Raw-ishDf.csv")

In [23]:
X = df.copy()
X["LabelA"]=df2["LabelA"]
y = X.pop("LabelA")


In [24]:
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler 
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers


In [25]:
# Building the Input Layer
input_layer = Input(shape =(X.shape[1], ))
  
# Building the Encoder network
encoded = Dense(100, activation ='tanh',
                activity_regularizer = regularizers.l1(10e-5))(input_layer)
encoded = Dense(50, activation ='tanh',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(25, activation ='tanh',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(12, activation ='tanh',
                activity_regularizer = regularizers.l1(10e-5))(encoded)
encoded = Dense(6, activation ='relu')(encoded)
  
# Building the Decoder network
decoded = Dense(12, activation ='tanh')(encoded)
decoded = Dense(25, activation ='tanh')(decoded)
decoded = Dense(50, activation ='tanh')(decoded)
decoded = Dense(100, activation ='tanh')(decoded)
  
# Building the Output Layer
output_layer = Dense(X.shape[1], activation ='relu')(decoded)


In [26]:
X_scaled = MinMaxScaler().fit_transform(X)
X_normal_scaled = X_scaled[y == 0]
X_fraud_scaled = X_scaled[y == 1]

In [27]:
# Defining the parameters of the Auto-encoder network
autoencoder = Model(input_layer, output_layer)
autoencoder.compile(optimizer ="adadelta", loss ="mse")
  
# Training the Auto-encoder network
autoencoder.fit(X_normal_scaled, X_normal_scaled, 
                batch_size = 16, epochs = 10, 
                shuffle = True, validation_split = 0.20)


Epoch 1/10
13661/13661 [==============================] - 21s 1ms/step - loss: 0.0771 - val_loss: 0.0711
Epoch 2/10
13661/13661 [==============================] - 21s 2ms/step - loss: 0.0587 - val_loss: 0.0668
Epoch 3/10
13661/13661 [==============================] - 21s 2ms/step - loss: 0.0531 - val_loss: 0.0654
Epoch 4/10
13661/13661 [==============================] - 20s 1ms/step - loss: 0.0510 - val_loss: 0.0627
Epoch 5/10
13661/13661 [==============================] - 21s 2ms/step - loss: 0.0492 - val_loss: 0.0601
Epoch 6/10
13661/13661 [==============================] - 21s 2ms/step - loss: 0.0473 - val_loss: 0.0513
Epoch 7/10
13661/13661 [==============================] - 21s 2ms/step - loss: 0.0441 - val_loss: 0.0468
Epoch 8/10
13661/13661 [==============================] - 20s 1ms/step - loss: 0.0428 - val_loss: 0.0432
Epoch 9/10
13661/13661 [==============================] - 20s 1ms/step - loss: 0.0419 - val_loss: 0.0399
Epoch 10/10
13661/13661 [==============================

In [28]:
hidden_representation = Sequential()
hidden_representation.add(autoencoder.layers[0])
hidden_representation.add(autoencoder.layers[1])
hidden_representation.add(autoencoder.layers[2])
hidden_representation.add(autoencoder.layers[3])
hidden_representation.add(autoencoder.layers[4])

In [29]:
# Separating the points encoded by the Auto-encoder as normal and fraud
normal_hidden_rep = hidden_representation.predict(X_normal_scaled)
fraud_hidden_rep = hidden_representation.predict(X_fraud_scaled)
  
# Combining the encoded points into a single table 
encoded_X = np.append(normal_hidden_rep, fraud_hidden_rep, axis = 0)
y_normal = np.zeros(normal_hidden_rep.shape[0])
y_fraud = np.ones(fraud_hidden_rep.shape[0])
encoded_y = np.append(y_normal, y_fraud)

In [30]:
# Splitting the encoded data for linear classification
X_train_encoded, X_test_encoded, y_train_encoded, y_test_encoded = train_test_split(encoded_X, encoded_y, test_size = 0.2)
  
# Splitting the original data for non-linear classification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)


In [31]:
# Building the logistic regression model
lrclf = LogisticRegression()
lrclf.fit(X_train_encoded, y_train_encoded)
  
# Storing the predictions of the linear model
y_pred_lrclf = lrclf.predict(X_test_encoded)
  
# Evaluating the performance of the linear model
print('Accuracy : '+str(accuracy_score(y_test_encoded, y_pred_lrclf)))


Accuracy : 0.9957024347878171


In [32]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics

X = encoded_X
y = encoded_y

dfInterno=pd.DataFrame()
scoring = {'accuracy', "precision", "recall",  "f1"}

from sklearn.ensemble import AdaBoostClassifier
# define the model with default hyperparameters
model = AdaBoostClassifier()
# define the grid of values to search
p_grid = dict()
p_grid['n_estimators'] = [100,200]
p_grid['learning_rate'] = [0.1,0.001,0.0001]


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X[train_index,:], X[test_index,:] 
    y_train,y_test= y[train_index], y[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)    

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9956612562789606
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9958634055359697
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9956275640471033
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.9959270479054109
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'learning_rate': 0.1, 'n_estimators': 200}
0.995665000335198


In [33]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\AdaboostEncodedGridSearchCVAtaques.csv", index = None, header=True) 

In [24]:
import pandas as pd
from sklearn.model_selection import cross_val_score
import sklearn.metrics as metrics
from sklearn.neural_network import MLPClassifier

X = encoded_X
y = encoded_y

dfInterno=pd.DataFrame()
scoring = {'accuracy', "precision", "recall",  "f1"}

from sklearn.ensemble import AdaBoostClassifier
# define the model with default hyperparameters
model = clf = MLPClassifier(alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)
# define the grid of values to search
p_grid = dict()
p_grid['alpha'] = [1e-5, 1e-3]
p_grid['max_iter'] = [100,200]


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X[train_index,:], X[test_index,:] 
    y_train,y_test= y[train_index], y[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)    

Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'alpha': 1e-05, 'max_iter': 100}
0.9991052985774138
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'alpha': 0.001, 'max_iter': 100}
0.9991726822002877
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'alpha': 1e-05, 'max_iter': 100}
0.9991689381440505
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'alpha': 1e-05, 'max_iter': 100}
0.9991502209459495
Fitting 5 folds for each of 4 candidates, totalling 20 fits
{'alpha': 1e-05, 'max_iter': 100}
0.9991389896180796


In [22]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\MLPEncodedGridSearchCV.csv", index = None, header=True) 

In [27]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics as metrics

X = encoded_X
y = encoded_y

dfInterno=pd.DataFrame()
# define the model with default hyperparameters
model=RandomForestClassifier(random_state=42)
scoring = {'accuracy', "precision", "recall",  "f1"}

# define the grid of values to search
p_grid = { 
    'n_estimators': [100, 200],
    #Empirical good default values [...] max_features=sqrt(n_features) for classification tasks
   # 'max_features': ["auto", "sqrt"],
    'max_depth' : [3,4,5],
    #'criterion' :['gini', 'entropy']
}

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):
    X_train,X_test= X[train_index,:], X[test_index,:] 
    y_train,y_test= y[train_index], y[test_index]
        
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    pipe= make_pipeline(MinMaxScaler(),StandardScaler(),grid)
    fittedgrid=pipe.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)  

Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 100}
0.997708962708179
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 100}
0.9977351682990345
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 100}
0.9976827582384452
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 200}
0.9976490681086908
Fitting 5 folds for each of 6 candidates, totalling 30 fits
{'max_depth': 5, 'n_estimators': 200}
0.9976865032055937


In [28]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\RFEncodedGridSearchCV.csv", index = None, header=True) 

In [36]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
import numpy as np
from tensorflow.keras.layers import Input, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers

X = encoded_X
y = encoded_y

# Function to create model, required for KerasClassifier
# Initialising the CNN
def create_model():
    model = Sequential()
    model.add(Conv1D(filters=31, kernel_size=3, activation='relu'))
    #model.add(Conv1D(filters=31, kernel_size=3, activation='relu'))
    model.add(Dropout(0.2))
    model.add(MaxPooling1D(pool_size=3))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid")) # binary activation output
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model



model = KerasClassifier(build_fn = create_model)


# define the grid search parameters
batch_size = [32, 64]
#optimizer= ["RMSprop", "adam"]
epochs = [10, 20]
#dropout = [0, 0.1, 0.2]
p_grid = dict(batch_size=batch_size, epochs=epochs)
scoring = {'accuracy', "precision", "recall",  "f1"}
dfInterno=pd.DataFrame()


outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)
# Nested CV with parameter optimization
for train_index, test_index in outer_cv.split(X,y):

    X_train,X_test= X[train_index,:], X[test_index,:] 
    y_train,y_test= y[train_index], y[test_index]
    
    X_train=MinMaxScaler().fit_transform(X_train)
    X_train=MinMaxScaler().fit_transform(X_train)

    X_test=MinMaxScaler().fit_transform(X_test)
    X_test=MinMaxScaler().fit_transform(X_test)

    #X_train=X_train.to_numpy()
    X_train = X_train[..., np.newaxis]
    #X_test=X_test.to_numpy()
    X_test = X_test[..., np.newaxis]
    
    grid = GridSearchCV(model,return_train_score=True,scoring=scoring, refit="accuracy",param_grid=p_grid, n_jobs=-1, cv= StratifiedKFold(n_splits=5, shuffle=True, random_state=1), verbose=4)
    fittedgrid=grid.fit(X_train, y_train)
#Saving the 25 subresults
    dfCurrent= pd.DataFrame(grid.cv_results_)
    dfInterno = pd.concat([dfInterno, dfCurrent], axis=0)
#Saving the best params and the results
    print(grid.best_params_)
    print(grid.best_score_)
    #dfBest= pd.concat([dfBest, dfCurrentBest], axis=0)      
    

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.1333 - accuracy: 0.9579
Epoch 2/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0511 - accuracy: 0.9849
Epoch 3/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0411 - accuracy: 0.9875
Epoch 4/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0360 - accuracy: 0.9892
Epoch 5/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0346 - accuracy: 0.9899
Epoch 6/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0331 - accuracy: 0.9903
Epoch 7/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0326 - accuracy: 0.9906
Epoch 8/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0315 - accuracy: 0.9909
Epoch 9/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0312 - accuracy: 0.9908
Epoch 10/20
8

8348/8348 [==============================] - 12s 1ms/step - loss: 0.0307 - accuracy: 0.9919
Epoch 17/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0305 - accuracy: 0.9918
Epoch 18/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0299 - accuracy: 0.9920
Epoch 19/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0291 - accuracy: 0.9923
Epoch 20/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0289 - accuracy: 0.9925
{'batch_size': 32, 'epochs': 20}
0.9970800555747072
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Epoch 1/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.1335 - accuracy: 0.9554
Epoch 2/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0504 - accuracy: 0.9830
Epoch 3/20
8348/8348 [==============================] - 12s 1ms/step - loss: 0.0418 - accuracy: 0.9860
Epoch 4/20
8348/8348 [==============================] - 12s 1ms/step - 

In [37]:
dfInterno.to_csv(r"C:\Users\White\Desktop\tfg\generados\CNNEncodedGridSearchCVAtaques.csv", index = None, header=True) 